## Istraživanje K-Means klasteriranja koristeći R i principe urednih podataka.

### [**Kviz prije predavanja**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

U ovoj lekciji naučit ćete kako kreirati klastere koristeći paket Tidymodels i druge pakete iz R ekosustava (nazvat ćemo ih prijateljima 🧑‍🤝‍🧑), te nigerijski glazbeni skup podataka koji ste ranije uvezli. Pokrit ćemo osnove K-Means metode za klasteriranje. Imajte na umu da, kao što ste naučili u prethodnoj lekciji, postoji mnogo načina za rad s klasterima, a metoda koju koristite ovisi o vašim podacima. Isprobat ćemo K-Means jer je to najčešće korištena tehnika klasteriranja. Krenimo!

Pojmovi koje ćete naučiti:

-   Silhouette ocjenjivanje

-   Metoda lakta

-   Inercija

-   Varijanca

### **Uvod**

[K-Means klasteriranje](https://wikipedia.org/wiki/K-means_clustering) je metoda koja potječe iz područja obrade signala. Koristi se za podjelu i grupiranje podataka u `k klastera` na temelju sličnosti njihovih značajki.

Klasteri se mogu vizualizirati kao [Voronoi dijagrami](https://wikipedia.org/wiki/Voronoi_diagram), koji uključuju točku (ili 'sjeme') i njezinu odgovarajuću regiju.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografika autorice Jen Looper</figcaption>


K-Means klasteriranje ima sljedeće korake:

1.  Data scientist započinje određivanjem željenog broja klastera koji će se kreirati.

2.  Zatim algoritam nasumično odabire K opažanja iz skupa podataka koja će služiti kao početni centri za klastere (tj. centroidi).

3.  Svako od preostalih opažanja dodjeljuje se svom najbližem centru.

4.  Zatim se izračunava novi prosjek svakog klastera i centroid se pomiče prema tom prosjeku.

5.  Sada kada su centri ponovno izračunati, svako opažanje se ponovno provjerava kako bi se vidjelo može li biti bliže drugom klasteru. Svi objekti se ponovno dodjeljuju koristeći ažurirane prosjeke klastera. Koraci dodjele klastera i ažuriranja centroida iterativno se ponavljaju dok se dodjele klastera ne prestanu mijenjati (tj. kada se postigne konvergencija). Obično algoritam završava kada svaka nova iteracija rezultira zanemarivim pomicanjem centroida i klasteri postanu statični.

<div>

> Imajte na umu da zbog nasumičnosti početnih k opažanja koja se koriste kao početni centroidi, možemo dobiti malo različite rezultate svaki put kada primijenimo postupak. Iz tog razloga, većina algoritama koristi nekoliko *nasumičnih početaka* i odabire iteraciju s najnižim WCSS. Stoga se snažno preporučuje uvijek pokretati K-Means s nekoliko vrijednosti *nstart* kako biste izbjegli *nepoželjan lokalni optimum.*

</div>

Ova kratka animacija koristeći [ilustracije](https://github.com/allisonhorst/stats-illustrations) Allison Horst objašnjava proces klasteriranja:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Ilustracija autorice @allison_horst</figcaption>



Osnovno pitanje koje se postavlja kod klasteriranja je sljedeće: kako znati na koliko klastera podijeliti svoje podatke? Jedan nedostatak korištenja K-Means metode uključuje činjenicu da ćete morati odrediti `k`, odnosno broj `centroida`. Srećom, `metoda lakta` pomaže u procjeni dobre početne vrijednosti za `k`. Uskoro ćete je isprobati.

### 

**Preduvjeti**

Nastavit ćemo točno tamo gdje smo stali u [prethodnoj lekciji](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), gdje smo analizirali skup podataka, napravili mnogo vizualizacija i filtrirali skup podataka na relevantna opažanja. Obavezno je provjerite!

Trebat će nam neki paketi za dovršavanje ovog modula. Možete ih instalirati pomoću: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternativno, skripta ispod provjerava imate li potrebne pakete za dovršavanje ovog modula i instalira ih za vas u slučaju da neki nedostaju.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Krenimo odmah!

## 1. Ples s podacima: Suzi na 3 najpopularnija glazbena žanra

Ovo je podsjetnik na ono što smo radili u prethodnoj lekciji. Hajdemo malo analizirati podatke!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 To je prošlo dobro!

## 2. Više istraživanja podataka.

Koliko su ovi podaci čisti? Provjerimo odstupanja pomoću boxplotova. Usredotočit ćemo se na numeričke stupce s manje odstupanja (iako biste mogli očistiti odstupanja). Boxplotovi mogu prikazati raspon podataka i pomoći u odabiru stupaca koje ćemo koristiti. Napomena, boxplotovi ne prikazuju varijancu, važan element za dobro grupirane podatke. Molimo pogledajte [ovu raspravu](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) za dodatno čitanje.

[Boxplotovi](https://en.wikipedia.org/wiki/Box_plot) koriste se za grafički prikaz distribucije `numeričkih` podataka, pa krenimo s *odabirom* svih numeričkih stupaca uz popularne glazbene žanrove.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Vidite kako selektor `where` olakšava ovaj zadatak 💁? Istražite i druge slične funkcije [ovdje](https://tidyselect.r-lib.org/).

Budući da ćemo izrađivati boxplot za svaku numeričku značajku i želimo izbjeći korištenje petlji, preoblikujmo naše podatke u *duži* format koji će nam omogućiti korištenje `facets` - podgrafova koji svaki prikazuju jedan podskup podataka.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Mnogo duže! Sada je vrijeme za neke `ggplot` grafove! Koji ćemo `geom` koristiti?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Sad je jasno da su podaci pomalo neuredni: promatrajući svaki stupac kao boxplot, možete vidjeti odstupanja. Mogli biste proći kroz skup podataka i ukloniti ta odstupanja, ali to bi učinilo podatke prilično oskudnima.

Za sada, odabrat ćemo koje stupce ćemo koristiti za našu vježbu grupiranja. Odabrat ćemo numeričke stupce sa sličnim rasponima. Mogli bismo kodirati `artist_top_genre` kao numerički, ali za sada ćemo ga izostaviti.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Izračunavanje k-means klasteriranja u R-u

K-means klasteriranje možemo izračunati u R-u pomoću ugrađene funkcije `kmeans`, pogledajte `help("kmeans()")`. Funkcija `kmeans()` prihvaća podatkovni okvir sa svim numeričkim stupcima kao svoj primarni argument.

Prvi korak pri korištenju k-means klasteriranja je odrediti broj klastera (k) koji će biti generirani u konačnom rješenju. Znamo da postoje 3 glazbena žanra koja smo izdvojili iz skupa podataka, pa pokušajmo s 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Objekt kmeans sadrži nekoliko informacija koje su dobro objašnjene u `help("kmeans()")`. Za sada ćemo se usredotočiti na nekoliko njih. Vidimo da su podaci grupirani u 3 klastera veličina 65, 110, 111. Izlaz također sadrži središta klastera (sredine) za 3 grupe kroz 5 varijabli.

Vektor klastera je dodjela klastera za svaku opažanje. Koristimo funkciju `augment` kako bismo dodali dodjelu klastera izvornom skupu podataka.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Savršeno, upravo smo podijelili naš skup podataka u 3 grupe. Dakle, koliko je dobro naše grupiranje 🤷? Pogledajmo `Silhouette score`.

### **Silhouette score**

[Silhouette analiza](https://en.wikipedia.org/wiki/Silhouette_(clustering)) može se koristiti za proučavanje udaljenosti između rezultirajućih klastera. Ova ocjena varira od -1 do 1, a ako je ocjena blizu 1, klaster je gust i dobro odvojen od drugih klastera. Vrijednost blizu 0 predstavlja preklapajuće klastere sa uzorcima vrlo blizu granice odluke susjednih klastera. [izvor](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Metoda prosječnog silueta izračunava prosječni siluet za opažanja za različite vrijednosti *k*. Visoka prosječna ocjena silueta ukazuje na dobro grupiranje.

Funkcija `silhouette` u paketu za klasteriranje koristi se za izračunavanje prosječne širine silueta.

> Silueta se može izračunati pomoću bilo koje [udaljenosti](https://en.wikipedia.org/wiki/Distance "Distance"), poput [Euklidske udaljenosti](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") ili [Manhattanske udaljenosti](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") o kojima smo raspravljali u [prethodnoj lekciji](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Naš rezultat je **.549**, što je točno u sredini. To ukazuje da naši podaci nisu osobito prikladni za ovu vrstu grupiranja. Pogledajmo možemo li ovu pretpostavku potvrditi vizualno. Paket [factoextra](https://rpkgs.datanovia.com/factoextra/index.html) pruža funkcije (`fviz_cluster()`) za vizualizaciju grupiranja.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Preklapanje u klasterima ukazuje na to da naši podaci nisu osobito prikladni za ovu vrstu klasterizacije, ali idemo dalje.

## 4. Određivanje optimalnog broja klastera

Osnovno pitanje koje se često postavlja kod K-Means klasterizacije je ovo - bez poznatih oznaka klasa, kako znati u koliko klastera podijeliti podatke?

Jedan od načina na koji to možemo pokušati saznati je korištenje uzorka podataka za `stvaranje serije modela klasterizacije` s povećavajućim brojem klastera (npr. od 1 do 10) i procjenu metrika klasterizacije poput **Silhouette score.**

Odredimo optimalan broj klastera izračunavanjem algoritma klasterizacije za različite vrijednosti *k* i procjenom **Sume kvadrata unutar klastera** (WCSS). Ukupna suma kvadrata unutar klastera (WCSS) mjeri kompaktnost klasterizacije, a želimo da bude što manja, pri čemu niže vrijednosti znače da su točke podataka bliže.

Istražimo učinak različitih izbora `k`, od 1 do 10, na ovu klasterizaciju.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Sada kada imamo ukupnu unutar-klastersku sumu kvadrata (tot.withinss) za svaki algoritam grupiranja s centrom *k*, koristimo [metodu lakta](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) kako bismo pronašli optimalan broj klastera. Metoda se sastoji od prikazivanja WCSS-a kao funkcije broja klastera i odabira [lakta krivulje](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Lakat krivulje") kao broja klastera koji ćemo koristiti.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Graf prikazuje značajno smanjenje WCSS-a (što znači veću *zbijenost*) kako se broj klastera povećava s jednog na dva, te dodatno primjetno smanjenje s dva na tri klastera. Nakon toga, smanjenje je manje izraženo, što rezultira `laktom` 💪 na grafikonu oko tri klastera. Ovo je dobar pokazatelj da postoje dva do tri razumno dobro odvojena klastera podataka.

Sada možemo nastaviti i izdvojiti model klasteriranja gdje je `k = 3`:

> `pull()`: koristi se za izdvajanje jednog stupca
>
> `pluck()`: koristi se za indeksiranje struktura podataka poput lista


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Odlično! Hajdemo vizualizirati dobivene klastere. Zanima li vas malo interaktivnosti pomoću `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Možda bismo očekivali da svaki klaster (predstavljen različitim bojama) ima različite žanrove (predstavljene različitim oblicima).

Pogledajmo točnost modela.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Točnost ovog modela nije loša, ali nije ni sjajna. Možda podaci nisu prikladni za K-Means klasteriranje. Ovi podaci su previše neuravnoteženi, premalo povezani i postoji prevelika varijanca između vrijednosti stupaca da bi se dobro grupirali. Zapravo, klasteri koji se formiraju vjerojatno su snažno pod utjecajem ili iskrivljeni zbog tri kategorije žanrova koje smo ranije definirali.

Unatoč tome, ovo je bio prilično poučan proces!

U dokumentaciji Scikit-learn-a možete vidjeti da model poput ovog, s klasterima koji nisu jasno definirani, ima problem s 'varijancom':

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografika iz Scikit-learn</figcaption>



## **Varijanca**

Varijanca se definira kao "prosjek kvadrata razlika od srednje vrijednosti" [izvor](https://www.mathsisfun.com/data/standard-deviation.html). U kontekstu ovog problema klasteriranja, odnosi se na podatke kod kojih vrijednosti našeg skupa podataka imaju tendenciju previše odstupati od srednje vrijednosti.

✅ Ovo je sjajan trenutak da razmislite o svim načinima na koje biste mogli ispraviti ovaj problem. Možete li još malo prilagoditi podatke? Koristiti različite stupce? Isprobati drugačiji algoritam? Savjet: Pokušajte [skalirati svoje podatke](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) kako biste ih normalizirali i testirali druge stupce.

> Isprobajte ovaj '[kalkulator varijance](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' kako biste bolje razumjeli koncept.

------------------------------------------------------------------------

## **🚀Izazov**

Provedite neko vrijeme s ovim bilježnicom, prilagođavajući parametre. Možete li poboljšati točnost modela dodatnim čišćenjem podataka (na primjer, uklanjanjem odstupanja)? Možete koristiti težine kako biste dali veću važnost određenim uzorcima podataka. Što još možete učiniti kako biste stvorili bolje klastere?

Savjet: Pokušajte skalirati svoje podatke. U bilježnici postoji zakomentirani kod koji dodaje standardno skaliranje kako bi stupci podataka više nalikovali jedni drugima u smislu raspona. Primijetit ćete da, iako se silueta smanjuje, 'pregib' na grafu lakta postaje glađi. To je zato što ostavljanje podataka neskaliranima omogućuje podacima s manjom varijancom da imaju veću težinu. Pročitajte više o ovom problemu [ovdje](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Kviz nakon predavanja**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Pregled i samostalno učenje**

-   Pogledajte simulator za K-Means [kao što je ovaj](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Ovaj alat možete koristiti za vizualizaciju uzoraka podataka i određivanje njihovih centroida. Možete uređivati nasumičnost podataka, broj klastera i broj centroida. Pomaže li vam ovo da steknete ideju o tome kako se podaci mogu grupirati?

-   Također, pogledajte [ovaj priručnik o K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) sa Stanforda.

Želite li isprobati svoje novo stečene vještine klasteriranja na skupovima podataka koji su prikladni za K-Means klasteriranje? Pogledajte:

-   [Treniranje i evaluacija modela klasteriranja](https://rpubs.com/eR_ic/clustering) koristeći Tidymodels i slične alate

-   [Analiza klastera K-Means](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-Means klasteriranje s principima urednih podataka](https://www.tidymodels.org/learn/statistics/k-means/)

## **Zadatak**

[Isprobajte različite metode klasteriranja](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## HVALA:

[Jen Looper](https://www.twitter.com/jenlooper) za stvaranje originalne Python verzije ovog modula ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) za stvaranje nevjerojatnih ilustracija koje čine R pristupačnijim i zanimljivijim. Pronađite više ilustracija u njenoj [galeriji](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Sretno u učenju,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Ilustracija @allison_horst</figcaption>



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije proizašle iz korištenja ovog prijevoda.
